## Análise dos dados

A fonte de dados para o desenvolvimento desse projeto são dois arquivos csv, cada um contendo os dados de sortimento para cada mês do período a ser analisado. 

In [48]:
# Importando bibliotecas que vão ser utilizadas
import pandas as pd
import numpy as np

In [49]:
# Importando os dados
df_maio = pd.read_csv("data/maio19.csv",delimiter=';', encoding='latin1')
df_junho = pd.read_csv("data/junho19.csv",delimiter=';', encoding='latin1')

In [50]:
# Unificando as bases de dados
df_junho.rename(columns={'VENDA BRUTA MENSAL JUNHO': 'VENDA BRUTA MENSAL'}, inplace=True)

dataframe = pd.concat([df_maio, df_junho], ignore_index=True)

In [52]:
# Renomenado as colunas
dataframe.rename(columns={'PLU': 'cod_plu',
                          'Loja': 'cod_loja',
                          'Região': 'regiao',
                          'Nome PLU': 'plu',
                          'Nome Depto': 'departamento',
                          'Nome Cat': 'categoria',
                          'Nome Subcat': 'subcategoria',
                          'Nome Loja': 'loja',
                          'Mês Referencia': 'mes_ano',
                          'Bandeira': 'bandeira',
                          'VENDA BRUTA MENSAL': 'venda_bruta'},
                    inplace=True)

### Tratando os dados

In [53]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44691 entries, 0 to 44690
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cod_plu       44691 non-null  int64 
 1   cod_loja      44691 non-null  int64 
 2   regiao        44691 non-null  object
 3   plu           44691 non-null  object
 4   departamento  44691 non-null  object
 5   categoria     44691 non-null  object
 6   subcategoria  44691 non-null  object
 7   loja          44691 non-null  object
 8   mes_ano       44691 non-null  object
 9   bandeira      44691 non-null  object
 10  venda_bruta   44691 non-null  object
dtypes: int64(2), object(9)
memory usage: 3.8+ MB


In [54]:
# Separando a coluna ano e mês em duas novas colunas
dataframe.mes_ano = pd.to_datetime(dataframe.mes_ano)
dataframe['ano'] = dataframe.mes_ano.dt.year
dataframe['mes'] = dataframe.mes_ano.dt.month_name()

# Renomeando os nomes dos meses
dataframe.mes.replace({'May': 'Maio', 'June': 'Junho'}, inplace=True)

#### Tratando valores de venda bruta

In [55]:
# Tratando valores de venda bruta
dataframe['venda_bruta_original'] = dataframe.venda_bruta

for index in dataframe.index:
    valor = str(dataframe.loc[index,  'venda_bruta'])

    if '.' in valor or ',' in valor:

        dataframe.loc[index,  'venda_bruta'] = valor.replace('.', '').replace(',', '.')

dataframe['venda_bruta'] = pd.to_numeric(dataframe.venda_bruta, errors='coerce')

In [56]:
subset = dataframe.shape[0]
subset_missing_values = dataframe.loc[dataframe.venda_bruta.isna()].shape[0]

print("{} linhas com valores faltantes, o equivalente a {:.2f}% da base de dados".format(subset_missing_values, (subset_missing_values/subset)*100))

9745 linhas com valores faltantes, o equivalente a 21.81% da base de dados


In [57]:
dataframe.venda_bruta_original.loc[dataframe.venda_bruta.isna()].value_counts()

 -       9745
Name: venda_bruta_original, dtype: int64

In [58]:
dataframe.venda_bruta.fillna(0, inplace=True)

In [59]:
dataframe.venda_bruta_original.loc[dataframe.venda_bruta < 0].value_counts()

-16     1
-22     1
-101    1
-1      1
-24     1
Name: venda_bruta_original, dtype: int64

In [60]:
subset = dataframe.shape[0]
subset_missing_values = dataframe.loc[dataframe.venda_bruta < 0].shape[0]

print("{} linhas com valores faltantes, o equivalente a {:.2f}% da base de dados".format(subset_missing_values, (subset_missing_values/subset)*100))

5 linhas com valores faltantes, o equivalente a 0.01% da base de dados


In [61]:
dataframe = dataframe.drop(index=dataframe[dataframe['venda_bruta'] < 0].index)


Após tratar os valores de venda bruta, convertendo esses dados para números decimais, foi obtido uma quantidade significante de valores faltantes que representava 21,81% da base de dados. Analisando os valores originais antes do tratamento, foi verificado que o valor de venda para esses itens estava preenchido com "-".<BR>
Como estamos analisando dados referentes a venda mensal desses produtos, a partir do dado preenchido originalmente pode-se inferir que ao invés desse problema representar falta de informação, ele representa que o valor de venda bruta mensal desses produtos é R$ 0,00.<BR>
<BR>
Os dados de venda bruta, também continham valores negativos, esses dados representavam 0.01% da base. Existe as hipóteses desses valores serem resultados de erros de registro e lançamento dos dados, ou também de que os valores negativos representem devoluções de compras. No entanto, devido a incapacidade de investigar o motivo para tal preenchimento da informação e considerando que o total desses dados sujos possuem pouca expressividade em relação a base, foi decidido remover esses dados para evitar análises enviesadas.

### Entendendo os dados presentes nos outros campos

In [62]:
dataframe.departamento.unique()

array(['PERECIVEIS', 'MERCEARIA', 'BAZAR', 'MUNDO FARMACIA', 'TEXTIL',
       'ELETRO-ELETRONICOS'], dtype=object)

In [63]:
dataframe.categoria.unique()

array(['PADARIA E ROTISSERIE', 'CARNES', 'PEREC COMPLEMENTAR', 'PEIXARIA',
       'FLV', 'PERFUMARIA', 'LIQUIDA', 'MERC COMPLEMENTAR',
       'ARRUMACAO E FVM', 'LIMPEZA/DESCARTAVEIS', 'CASA', 'PET E JARDIM',
       'MERCEARIA BASICA', 'CULTURA', 'DROGARIAS', 'CAMEBA', 'TELEFONIA',
       'MOVEIS E DECORACAO', 'ACESSORIOS / SOFTWAR'], dtype=object)

In [64]:
dataframe.subcategoria.unique()

array(['PADARIA', 'BOVINOS', 'SUINOS E OUTROS', 'QUEIJOS', 'AVES',
       'QUEIJOS MANIPULADOS', 'ROTISSERIE', 'PEIXES FRESCOS',
       'FOLHAGENS E PROCESS', 'FRIOS MANIPULADOS', 'FRUTAS',
       'SALGADOS E DEFUMADOS', 'LEGUMES', 'BACALHAU', 'PEIXES CONGELADOS',
       'CAFE/LANCHONETE', 'CUIDADO COM CORPO', 'NAO ALCOOLICOS',
       'DOCES E SOBREMESAS', 'FACA VOCE MESMO', 'VINHOS',
       'DESCART HIGIENICOS', 'DESCART CASA FESTA', 'BOMBONIERE',
       'BISCOITOS E TORRADAS', 'CEREAIS E SAUDAVEIS', 'LIMPEZA',
       'REFRIGERANTES', 'PADARIA INDUSTRIALI', 'ARTES DA MESA', 'PET',
       'HIGIENE ORAL', 'GRAOS E FARINACEOS', 'CAFE',
       'CHURRASCO E ARMAZEN', 'MASSAS SECAS', 'FRIOS',
       'LATICINIOS E MASSAS', 'ALCOOLICOS', 'CUIDADO COM ROSTO',
       'MERCEARIA SALGADA', 'SALGADINHOS/APERIT', 'LACTEOS E ACHOCOLAT',
       'UTENSILIOS LIMPEZA', 'ARROZ E FEIJAO', 'COZINHAR E ASSAR',
       'OLEOS E AZEITES', 'LIVROS REVISTAS NAC', 'CIGARROS', 'ARRUMACAO',
       'SOBREMESAS CONGE

In [65]:
dataframe.bandeira.unique()

array(['PA'], dtype=object)

In [66]:
lista_plu = dataframe.plu.unique().tolist()
lista_plu.sort()
len(lista_plu)

22960

### Exportando arquivos para serem utilizados no PowerBI

In [70]:
# Exportando a base completa de sortimentos 
dataframe.drop(columns='venda_bruta_original', inplace=True)

dataframe.to_excel("data/base_sortimentos.xlsx", index=False)

In [115]:
# Criando arquivo auxiliar para calcular as entradas e saídas de sortimento no período
df_count_junho = dataframe[['mes', 'departamento', 'categoria', 'cod_plu']].groupby(['mes', 'departamento', 'categoria']).count().query("mes == 'Junho'").reset_index().rename(columns={'cod_plu': 'count_junho'})
df_count_maio = dataframe[['mes', 'departamento', 'categoria', 'cod_plu']].groupby(['mes', 'departamento', 'categoria']).count().query("mes == 'Maio'").reset_index().rename(columns={'cod_plu': 'count_maio'})

df_count = pd.merge(df_count_maio, df_count_junho, how='inner', left_on=['departamento', 'categoria'], right_on = ['departamento', 'categoria'])

df_count['diferenca'] = df_count.count_junho - df_count.count_maio

df_count.loc[df_count.diferenca < 0, 'qtde_saidas'] = df_count.diferenca * -1
df_count.loc[df_count.diferenca > 0, 'qtde_entradas'] = df_count.diferenca

df_count.to_excel("data/periodos_diff_sortimentos.xlsx", index=False)


In [120]:
# Criando arquivo auxiliar para calcular as categorias em aumento ou regressão de vendas
df_sum_junho = dataframe[['mes', 'categoria', 'venda_bruta']].groupby(['mes',  'categoria']).sum().query("mes == 'Junho'").reset_index().rename(columns={'venda_bruta': 'venda_bruta_junho'})
df_sum_maio = dataframe[['mes', 'categoria', 'venda_bruta']].groupby(['mes',  'categoria']).sum().query("mes == 'Maio'").reset_index().rename(columns={'venda_bruta': 'venda_bruta_maio'})

df_sum = pd.merge(df_sum_maio, df_sum_junho, how='inner', left_on=['categoria'], right_on = ['categoria'])

df_sum['diferenca'] = df_sum.venda_bruta_junho - df_sum.venda_bruta_maio

df_sum.loc[df_sum.diferenca < 0, 'tendencia_periodo'] = 'Regressão'
df_sum.loc[df_sum.diferenca > 0, 'tendencia_periodo'] = 'Aumento'

df_sum['categoria'].loc[df_sum.tendencia_periodo == 'Regressão']

dataframe.loc[dataframe.categoria == 'CASA', 'tendencia_periodo'] = 'Regressão'
dataframe.loc[dataframe.categoria == 'CULTURA', 'tendencia_periodo'] = 'Regressão'
dataframe.loc[dataframe.categoria == 'FLV', 'tendencia_periodo'] = 'Regressão'
dataframe.loc[dataframe.categoria == 'MOVEIS E DECORACAO', 'tendencia_periodo'] = 'Regressão'
dataframe.loc[dataframe.categoria == 'PEIXARIA', 'tendencia_periodo'] = 'Regressão'

dataframe.tendencia_periodo.fillna('Aumento', inplace=True)

dataframe.to_excel("data/base_sortimentos_tendencia.xlsx", index=False)